In [91]:
import pandas as pd

# Load the raw dataset
data_path = "../data/raw/MachineLearningRating_v3.txt"
df = pd.read_csv(data_path, sep='|')

# Check the first few rows and columns
print("Columns in dataset:", df.columns.tolist())
print(df.head())



C:\Users\It's Blue\AppData\Local\Temp\ipykernel_16556\3150359416.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, sep='|')


Columns in dataset: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']
   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0               145249     12827  2015-03-01 00:00:00             True   
1               145249     128

In [115]:
# Count missing values per column
missing_counts = df.isna().sum()
print("Columns with missing values:")
print(missing_counts[missing_counts > 0])


Columns with missing values:
Bank                        145961
AccountType                  40232
MaritalStatus                 8259
Gender                        9536
mmcode                         552
VehicleType                    552
make                           552
Model                          552
Cylinders                      552
cubiccapacity                  552
kilowatts                      552
bodytype                       552
NumberOfDoors                  552
VehicleIntroDate               552
CustomValueEstimate         779642
CapitalOutstanding               2
NewVehicle                  153295
WrittenOff                  641901
Rebuilt                     641901
Converted                   641901
CrossBorder                 999400
NumberOfVehiclesInFleet    1000098
dtype: int64


In [116]:
# Make sure 'TotalClaims' exists
if 'TotalClaims' in df.columns:
    df_claims = df[df['TotalClaims'] > 0].copy()
    print("Shape after filtering for claims > 0:", df_claims.shape)
else:
    raise KeyError("'TotalClaims' column not found in dataset!")


Shape after filtering for claims > 0: (2788, 52)


In [117]:
# Separate numeric and categorical columns
numeric_cols = df_claims.select_dtypes(include='number').columns
categorical_cols = df_claims.select_dtypes(include='object').columns

# Numeric imputation: fill with median
df_claims[numeric_cols] = df_claims[numeric_cols].fillna(df_claims[numeric_cols].median())

# Categorical imputation: fill with mode or 'Unknown'
for col in categorical_cols:
    df_claims[col] = df_claims[col].fillna('Unknown')


In [118]:
df_encoded = pd.get_dummies(df_claims, columns=categorical_cols, drop_first=True)

# Remove duplicate column names to avoid XGBoost errors
df_encoded = df_encoded.loc[:, ~df_encoded.columns.duplicated()]

print("Shape after encoding and removing duplicate columns:", df_encoded.shape)


Shape after encoding and removing duplicate columns: (2788, 640)


In [119]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop(['TotalClaims', 'CalculatedPremiumPerTerm', 'TotalPremium'], axis=1)
y = df_encoded['TotalClaims']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape, "X_test shape:", X_test.shape)


X_train shape: (2230, 637) X_test shape: (558, 637)


In [121]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop(['TotalClaims', 'CalculatedPremiumPerTerm', 'TotalPremium'], axis=1)
y = df_encoded['TotalClaims']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shapes
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)


X_train: (2230, 637) X_test: (558, 637)
y_train: (2230,) y_test: (558,)


In [122]:
# Keep only columns common to both
common_cols = X_train.columns.intersection(X_test.columns)

X_train = X_train[common_cols].copy()
X_test = X_test[common_cols].copy()

# Remove duplicates again just in case
X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_test = X_test[X_train.columns]  # align columns exactly


In [137]:
# Keep only columns in X_test that exist in X_train
X_test = X_test[X_train.columns]

# Now X_train and X_test have the same columns
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (2230, 637)
X_test shape: (558, 637)


In [138]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Force numeric columns to numeric (coerce errors to NaN)
X_train = X_train.apply(pd.to_numeric, errors='ignore')
X_test = X_test.apply(pd.to_numeric, errors='ignore')

# Now select numeric columns
num_cols = X_train.select_dtypes(include=['number']).columns
cat_cols = X_train.select_dtypes(exclude=['number']).columns
print("Numeric columns:", len(num_cols), "Categorical columns:", len(cat_cols))


Numeric columns: 12 Categorical columns: 625


C:\Users\It's Blue\AppData\Local\Temp\ipykernel_16556\2192661667.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  X_train = X_train.apply(pd.to_numeric, errors='ignore')
C:\Users\It's Blue\AppData\Local\Temp\ipykernel_16556\2192661667.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  X_test = X_test.apply(pd.to_numeric, errors='ignore')


In [139]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Convert columns to numeric where possible; leave others as-is
def force_numeric(df):
    for col in df.columns:
        # Try converting, invalid parsing becomes NaN automatically
        if pd.api.types.is_object_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

X_train = force_numeric(X_train)
X_test = force_numeric(X_test)

# Now select numeric and categorical columns
num_cols = X_train.select_dtypes(include=['number']).columns
cat_cols = X_train.select_dtypes(exclude=['number']).columns

print("Numeric columns:", len(num_cols), "Categorical columns:", len(cat_cols))


Numeric columns: 12 Categorical columns: 625


In [149]:
# Numeric
num_imputer = SimpleImputer(strategy='median')
X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

# Categorical
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])

# Verify
assert not X_train.isna().any().any(), "X_train still has NaNs!"
assert not X_test.isna().any().any(), "X_test still has NaNs!"
print("✅ All missing values handled. Ready for modeling!")


✅ All missing values handled. Ready for modeling!


In [141]:
# First, verify what columns actually exist
print("X_train columns:", X_train.columns.tolist())
print("num_cols:", num_cols)
print("cat_cols:", cat_cols)

# Get only columns that actually exist
num_cols_exist = [col for col in num_cols if col in X_train.columns]
cat_cols_exist = [col for col in cat_cols if col in X_train.columns]

# Update your lists
num_cols = num_cols_exist
cat_cols = cat_cols_exist

print(f"Found {len(num_cols)} numeric columns, {len(cat_cols)} categorical columns")

X_train columns: ['UnderwrittenCoverID', 'PolicyID', 'IsVATRegistered', 'PostalCode', 'mmcode', 'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CustomValueEstimate', 'NumberOfVehiclesInFleet', 'SumInsured', 'TransactionMonth_2013-12-01 00:00:00', 'TransactionMonth_2014-01-01 00:00:00', 'TransactionMonth_2014-02-01 00:00:00', 'TransactionMonth_2014-03-01 00:00:00', 'TransactionMonth_2014-04-01 00:00:00', 'TransactionMonth_2014-05-01 00:00:00', 'TransactionMonth_2014-06-01 00:00:00', 'TransactionMonth_2014-07-01 00:00:00', 'TransactionMonth_2014-08-01 00:00:00', 'TransactionMonth_2014-09-01 00:00:00', 'TransactionMonth_2014-10-01 00:00:00', 'TransactionMonth_2014-11-01 00:00:00', 'TransactionMonth_2014-12-01 00:00:00', 'TransactionMonth_2015-01-01 00:00:00', 'TransactionMonth_2015-02-01 00:00:00', 'TransactionMonth_2015-03-01 00:00:00', 'TransactionMonth_2015-04-01 00:00:00', 'TransactionMonth_2015-05-01 00:00:00', 'TransactionMonth_2015-06-01 00:00:00', 

In [144]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# First, let's inspect your data
print("DataFrame info:")
print(X_train.info())
print("\nMissing values per column:")
print(X_train.isnull().sum())

# Get column types
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
boolean_cols = X_train.select_dtypes(include=['bool']).columns.tolist()

print(f"\nNumeric columns ({len(numeric_cols)}): {numeric_cols}")
print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")
print(f"Boolean columns ({len(boolean_cols)}): {boolean_cols}")

# Check for all-NaN columns
all_nan_cols = X_train.columns[X_train.isnull().all()].tolist()
print(f"\nColumns with all NaN values: {all_nan_cols}")

# Remove 'NumberOfVehiclesInFleet' if it's all NaN
if 'NumberOfVehiclesInFleet' in numeric_cols:
    numeric_cols.remove('NumberOfVehiclesInFleet')
    print(f"Removed 'NumberOfVehiclesInFleet' from numeric columns - all NaN")

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 2230 entries, 191385 to 281696
Columns: 637 entries, UnderwrittenCoverID to Product_Mobility Metered Taxis: Monthly
dtypes: bool(625), float64(8), int64(4)
memory usage: 1.6 MB
None

Missing values per column:
UnderwrittenCoverID                           0
PolicyID                                      0
IsVATRegistered                               0
PostalCode                                    0
mmcode                                        0
                                             ..
CoverGroup_Motor Comprehensive                0
Section_Motor Comprehensive                   0
Section_Optional Extended Covers              0
Product_Mobility Commercial Cover: Monthly    0
Product_Mobility Metered Taxis: Monthly       0
Length: 637, dtype: int64

Numeric columns (12): ['UnderwrittenCoverID', 'PolicyID', 'PostalCode', 'mmcode', 'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CustomValue

In [145]:
# Handle numeric columns (excluding all-NaN columns)
if numeric_cols:
    num_imputer = SimpleImputer(strategy='median')
    
    # Check which numeric columns actually have some data
    numeric_cols_with_data = [col for col in numeric_cols 
                             if not X_train[col].isnull().all()]
    
    if numeric_cols_with_data:
        X_train_num_imputed = num_imputer.fit_transform(X_train[numeric_cols_with_data])
        X_test_num_imputed = num_imputer.transform(X_test[numeric_cols_with_data])
        
        X_train[numeric_cols_with_data] = X_train_num_imputed
        X_test[numeric_cols_with_data] = X_test_num_imputed
        
        # For all-NaN columns, fill with 0 or drop them
        numeric_cols_all_nan = [col for col in numeric_cols 
                               if X_train[col].isnull().all()]
        if numeric_cols_all_nan:
            print(f"\nFilling all-NaN numeric columns with 0: {numeric_cols_all_nan}")
            X_train[numeric_cols_all_nan] = X_train[numeric_cols_all_nan].fillna(0)
            X_test[numeric_cols_all_nan] = X_test[numeric_cols_all_nan].fillna(0)
    else:
        print("No numeric columns with data to impute")

In [146]:
# Handle categorical columns (now including converted boolean columns)
if categorical_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    
    # Check which categorical columns have data
    cat_cols_with_data = [col for col in categorical_cols 
                         if not X_train[col].isnull().all()]
    
    if cat_cols_with_data:
        X_train_cat_imputed = cat_imputer.fit_transform(X_train[cat_cols_with_data])
        X_test_cat_imputed = cat_imputer.transform(X_test[cat_cols_with_data])
        
        X_train[cat_cols_with_data] = X_train_cat_imputed
        X_test[cat_cols_with_data] = X_test_cat_imputed
        
        # For all-NaN categorical columns
        cat_cols_all_nan = [col for col in categorical_cols 
                           if X_train[col].isnull().all()]
        if cat_cols_all_nan:
            print(f"\nFilling all-NaN categorical columns with 'missing': {cat_cols_all_nan}")
            X_train[cat_cols_all_nan] = X_train[cat_cols_all_nan].fillna('missing')
            X_test[cat_cols_all_nan] = X_test[cat_cols_all_nan].fillna('missing')
    else:
        print("No categorical columns with data to impute")

In [148]:
def handle_missing_values(X_train, X_test):
    """Handle missing values for different data types"""
    
    # Make copies to avoid SettingWithCopyWarning
    X_train_proc = X_train.copy()
    X_test_proc = X_test.copy()
    
    # Get columns by type
    numeric_cols = X_train_proc.select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_cols = X_train_proc.select_dtypes(include=['object', 'category']).columns.tolist()
    bool_cols = X_train_proc.select_dtypes(include=['bool']).columns.tolist()
    
    # 1. Handle boolean columns (convert to int first)
    for col in bool_cols:
        X_train_proc[col] = X_train_proc[col].astype('float')  # or 'int'
        X_test_proc[col] = X_test_proc[col].astype('float')
        # Add to numeric columns for imputation
        numeric_cols.append(col)
    
    # 2. Handle numeric columns
    if numeric_cols:
        # Remove columns that are all NaN
        numeric_cols_valid = [col for col in numeric_cols 
                            if not X_train_proc[col].isnull().all()]
        
        if numeric_cols_valid:
            num_imputer = SimpleImputer(strategy='median')
            X_train_proc[numeric_cols_valid] = num_imputer.fit_transform(
                X_train_proc[numeric_cols_valid])
            X_test_proc[numeric_cols_valid] = num_imputer.transform(
                X_test_proc[numeric_cols_valid])
        
        # Fill all-NaN columns with 0
        for col in set(numeric_cols) - set(numeric_cols_valid):
            X_train_proc[col] = 0
            X_test_proc[col] = 0
    
    # 3. Handle categorical columns
    if cat_cols:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        X_train_proc[cat_cols] = cat_imputer.fit_transform(X_train_proc[cat_cols])
        X_test_proc[cat_cols] = cat_imputer.transform(X_test_proc[cat_cols])
    
    # Verify
    print(f"\nMissing values after imputation:")
    print(f"X_train: {X_train_proc.isnull().sum().sum()}")
    print(f"X_test: {X_test_proc.isnull().sum().sum()}")
    
    return X_train_proc, X_test_proc

# Usage
X_train_processed, X_test_processed = handle_missing_values(X_train, X_test)
print("✅ All missing values handled!")


Missing values after imputation:
X_train: 0
X_test: 0
✅ All missing values handled!


In [147]:
# Convert boolean columns to object type first
bool_cols = X_train.select_dtypes(include=['bool']).columns.tolist()
for col in bool_cols:
    X_train[col] = X_train[col].astype('object')
    X_test[col] = X_test[col].astype('object')

# Update your cat_cols list
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Remove all-NaN columns from num_cols
if 'NumberOfVehiclesInFleet' in num_cols:
    num_cols.remove('NumberOfVehiclesInFleet')
    X_train['NumberOfVehiclesInFleet'] = 0  # or X_train.drop(columns=['NumberOfVehiclesInFleet'])
    X_test['NumberOfVehiclesInFleet'] = 0

# Now run your imputation
num_imputer = SimpleImputer(strategy='median')
X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])